<a href="https://colab.research.google.com/github/ShwetaBaranwal/HandsonML/blob/main/KFoldCrossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [5]:
iris = load_iris()
X = iris["data"][:, 3:]
y = (iris['target']==2).astype('int') # converting to binary class

In [6]:
X.shape, y.shape

((150, 1), (150,))

In [7]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

In [23]:
class KNeighborClassifier:
  def __init__(self, n_neighbor, model, X, y, scoring):
    self.K = n_neighbor
    self.model = model
    self.X = X
    self.y = y
    self.scoring = scoring

  def get_indexs(self):
    idx = np.random.permutation(len(self.X))
    d = int(len(self.X)/self.K)
    index_ = []
    for i in range(self.K):
      test_index = idx[d*i: d*(i+1)]
      train_index = np.concatenate([idx[:d*i], idx[d*(i+1):]])
      index_.append((train_index, test_index))

    return index_

  def fit(self):
    res = []
    y_cross_val_pred = np.empty(len(self.y))
    for train_idx, test_idx in self.get_indexs():
      X_train = self.X[train_idx]
      y_train = self.y[train_idx]
      X_test = self.X[test_idx]
      y_test = self.y[test_idx]
      self.model.fit(X_train, y_train)
      res.append(self.scoring(self.model.predict(X_test), y_test))
      y_cross_val_pred[test_idx] = self.model.predict_proba(X_test)[:,1]
    return res, y_cross_val_pred




In [24]:
knn=  KNeighborClassifier(3, log_reg, X, y, accuracy_score)
knn.fit()

([0.98, 0.92, 0.96],
 array([0.00427244, 0.00427244, 0.00475374, 0.00427244, 0.00475374,
        0.0096211 , 0.00572928, 0.0039449 , 0.0039449 , 0.00291352,
        0.00427244, 0.0039449 , 0.00291352, 0.00291352, 0.0039449 ,
        0.00831406, 0.00831406, 0.0062612 , 0.00572928, 0.0062612 ,
        0.00475374, 0.00831406, 0.00475374, 0.01340369, 0.00475374,
        0.00475374, 0.00831406, 0.00427244, 0.0039449 , 0.0039449 ,
        0.0039449 , 0.00831406, 0.00271474, 0.00475374, 0.00475374,
        0.00427244, 0.0039449 , 0.00271474, 0.0039449 , 0.0039449 ,
        0.00572928, 0.00676574, 0.00427244, 0.01937513, 0.00916718,
        0.0062612 , 0.00427244, 0.00475374, 0.00475374, 0.00427244,
        0.30131193, 0.38772801, 0.34143717, 0.19673663, 0.38772801,
        0.19163117, 0.48183632, 0.08488117, 0.19673663, 0.25265896,
        0.07366573, 0.32530091, 0.07366573, 0.30131193, 0.19673663,
        0.26272434, 0.38772801, 0.07366573, 0.32530091, 0.10438959,
        0.66723037, 0.19163

In [20]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X, y)
accuracy_score(np.where(log_reg.predict(X)<0.5, 0, 1), y)

0.96

In [34]:
idx = np.random.permutation(len(X))
K = 3
K_fold_idx = []

for k in range(K):

  idx_k = int(len(idx)/K)
  test_index = idx[idx_k*k: idx_k*(k+1)]
  train_index = np.concatenate([idx[:idx_k*k], idx[idx_k*(k+1):]])
  K_fold_idx.append((train_index, test_index))
K_fold_idx

[(array([131, 129,  67,  22, 106,  13,  28, 116,  51,  95, 132,  90,  68,
          72,  78,  84,   1,  73,  27,  58,  56, 127, 140,  47,  20, 112,
          98,  83, 117,  44,  61,  70,   5,  42,  43,  18,  26,  24,  94,
          96, 138,  87, 109, 123,  21, 145, 108, 114, 119,  93,  15, 120,
           7,  17, 100,  74,  14,  50, 146, 126, 139,  35,  80, 107,  99,
          41,  11,  36, 105,  91, 124,  81,  92,  39,  29,  79, 104,   9,
         141,  40, 135,  60, 101, 143, 133, 113,  23, 137,  45, 147,  88,
          97,  37, 122,  86,  62, 142, 149,  69,  89]),
  array([102, 111,  38,  66,  34,  49,  16,   2, 134,  52, 136,   4, 115,
          76, 144,  31,  64, 121, 130,  63,   0,  85,  48,  55,  32,  75,
          65,  33,   3,  71,  10,  25,   8, 125,  12,  30,  53,  19,  54,
          57,  59,   6, 110,  77, 128, 118, 148,  46,  82, 103])),
 (array([102, 111,  38,  66,  34,  49,  16,   2, 134,  52, 136,   4, 115,
          76, 144,  31,  64, 121, 130,  63,   0,  85,  48,  55,

In [37]:
score = []
y_cross_val_pred = np.empty(len(X))
for train_idx, test_idx in K_fold_idx:
  X_train, y_train = X[train_idx], y[train_idx]
  X_test, y_test = X[test_idx], y[test_idx]
  log_reg.fit(X_train, y_train)
  score.append(accuracy_score(np.where(log_reg.predict(X_test)<0.5, 0, 1), y_test))
  y_cross_val_pred[test_idx] = log_reg.predict(X_test)
print (score)
print (y_cross_val_pred)

[0.96, 0.94, 0.98]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
